In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
a = .1
sigma = .01
r_0 = .025
dt = 1/252
T_max = 5.0
time_grid = np.arange(0, T_max, dt)
n_paths = 100

In [ ]:
from dataclasses import dataclass

@dataclass
class HullWhiteParams:
  a: float
  sigma: float
  r_0: float
  dt: float
  T: int
  n_paths: int
  time_grid: np.ndarray


  def zero_rate(self, T):
    return 0.02 + 0.005 * (1-np.exp(-0.3*T))

  def P0T(self, T):
    return np.exp(-self.zero_rate(T)*T)

  def f0T(self, T=None):
    eps = 1e-5
    if T is None:
      T = self.T
    return -(np.log(self.P0T(T + eps))
                - np.log(self.P0T(T - eps))) / (2 * eps)

  def df0T_dt(self, t = None):
    eps = 1e-5
    if t is None:
      t = self.T
    return (self.f0T(t + eps) - self.f0T(t - eps)) / (2 * eps)

  def theta(self, t):
    return (
              self.df0T_dt(t)
              + self.a * self.f0T(t) 
              + (self.sigma**2 / (2*self.a**2))*(1-np.exp(-2 * self.a * t))
           )



In [ ]:
class HullWhiteSimulation(HullWhiteParams):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.r_i = None

    def trapezoidal_phi(self, t):
        return 0.5 * self.dt * (
            self.theta(t) * np.exp(-self.a * self.dt)
            + self.theta(t + self.dt)
        )

    def exact_short_rate(self):
        n_steps = len(self.time_grid)
        r = np.zeros((self.n_paths, n_steps))
        r[:, 0] = self.r_0

        Z = np.random.standard_normal((self.n_paths, n_steps))
        vol = self.sigma * np.sqrt((1 - np.exp(-2 * self.a * self.dt)) / (2 * self.a))

        for i in range(1, n_steps):
            r[:, i] = (
                r[:, i-1] * np.exp(-self.a * self.dt)
                + self.trapezoidal_phi(self.time_grid[i-1])
                + vol * Z[:, i]
            )

        self.r_i = r
        return r

    def plot_simulation(self):
        assert self.r_i is not None
        plt.plot(self.r_i.T, alpha=0.3)
        plt.title("Hull–White Short Rate Paths")
        plt.show()

In [ ]:
hullwhite = HullWhiteSimulation(a, sigma, r_0, dt, T_max, n_paths, time_grid)

In [ ]:
hullwhite.exact_short_rate()
hullwhite.plot_simulation()